In [ ]:
run_simulation_gamma <- function(n1,n0, beta1, M = 1000, B = 200) {
  library(birk); library(caret)
# Set c_opt values
  alpha0=1.5
  beta0=1
  mu0=alpha0*beta0
  alpha1=2.5
  beta1=beta1
  mu1= alpha1*beta1
  predefined_cz <- c(1.35,1.81,2.41,3.38)
  predefined_er <- c(1.38,1.82,2.36,3.24)
  predefined_youden <- c(1.12,1.79,2.45,3.42)
  predefined_maxchi<-c(0.80,1.73,2.54,3.51)
  predefined_iu <- c(1.42,1.78,2.41,3.30)
  idx <- which(c(0.79,1.22,1.97,3.82) == beta1)
  c_opt_cz <- predefined_cz[idx]
  c_opt_er <- predefined_er[idx]
  c_opt_youden <- predefined_youden[idx]
  c_opt_maxchi <- predefined_maxchi[idx]
  c_opt_iu <- predefined_iu[idx] 
  c_opt_ed <- ((n1 * mu1) + (n0* mu0)) / (n0 + n1)
  c_opt <- c(c_opt_cz, c_opt_er, c_opt_youden, c_opt_maxchi, c_opt_iu, c_opt_ed)
  
  cz = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$cz = rdf$se*rdf$sp
    rdf = data.frame(na.omit(rdf))
    maxcz = rdf[rdf$cz==max(rdf$cz),1]
    cz_cuts = c(maxcz)
    closest_indices_cz = which.closest(cz_cuts,c_opt_cz)
    closest_cz = cz_cuts[closest_indices_cz] 
    return(closest_cz)
  }
  er = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$fpr = 1-rdf$sp
    rdf$er = sqrt((rdf$fpr^2)+(rdf$se-1)^2)
    rdf = data.frame(na.omit(rdf))
    miner = rdf[rdf$er==min(rdf$er),1]
    er_cuts = c(miner)
    closest_indices_er = which.closest(er_cuts,c_opt_er)
    closest_er = er_cuts[closest_indices_er] 
    return(closest_er)
  }
  youden = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$youden = rdf$se+rdf$sp-1
    rdf = data.frame(na.omit(rdf))
    maxyouden = rdf[rdf$youden==max(rdf$youden),1]
    youden_cuts = c(maxyouden)
    closest_indices_y = which.closest(youden_cuts,c_opt_youden)
    closest_youden = youden_cuts[closest_indices_y]
    return(closest_youden)
  }
  index_euclidian = function(data){
    ###Index of Euclidian
    n=nrow(data)
    sum=numeric()
    dif=numeric()
    minimum=function(x){
      for(i in 1:n){
        for(j in 1:n){
          dif[j]=(x[i]-x[j])^2
          j=j+1
        }
        sum [i]=sum(dif)
        i=i+1
        
      }
      return(sum)
    }
    data$distance = minimum(data$X)
    ed = data[data$distance==min(data$distance),1]
    ed_cuts = c(ed)
    closest_indices_ed = which.closest(ed_cuts,c_opt_ed)
    closest_ed = ed_cuts[closest_indices_ed]
    return(closest_ed)
  }
  
  chi_max = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$fpr = 1-rdf$sp
    rdf$chi = ((rdf$se-rdf$fpr)^2)/
      (((totpos*rdf$se+totneg*rdf$fpr)/(totneg+totpos))*
         (1-(totpos*rdf$se+totneg*rdf$fpr)/(totneg+totpos))*(1/totpos+1/totneg))
    rdf = data.frame(na.omit(rdf))
    maxchi = rdf[rdf$chi==max(rdf$chi),1]
    maxchi_cuts= c(maxchi)
    closest_indices_maxchi = which.closest(maxchi_cuts,c_opt_maxchi)
    closest_maxchi = maxchi_cuts[closest_indices_maxchi]
    return(closest_maxchi)
    
  }
  index_union = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$iu = abs(rdf$se - rdf$sp)
    iu= rdf[rdf$iu==min(rdf$iu),1]
    iu_cuts = c(iu)
    closest_indices_iu = which.closest(iu_cuts,c_opt_iu)
    closest_iu = iu_cuts[closest_indices_iu]
    return(closest_iu)
  }
  opt_boot=matrix(nrow=B,ncol = 6)
  length=numeric()
  sd_boot=numeric()
  coverage = numeric()
  ub=numeric()
  lb=numeric()
  boot = function(data,B,true_param){
    for(i in 1:B){
      data_boot = data[sample(nrow(data),replace = T),]
      closest_cz_boot = cz(data_boot)
      closest_er_boot = er(data_boot)
      closest_youden_boot = youden(data_boot)
      closest_minp_boot = chi_max(data_boot)
      closest_ed_boot = index_euclidian(data_boot)
      closest_iu_boot = index_union(data_boot)
      opt_boot[i,1:6] = matrix(c(closest_cz_boot,closest_er_boot,closest_youden_boot,closest_minp_boot,
                                 closest_iu_boot,closest_ed_boot),ncol=6,byrow=F)
    }
    colnames(opt_boot)=c("cz","er","youden","minp","iu","ed")
    for(j in 1:6){
      ub[j] = quantile(opt_boot[,j],0.975)
      lb[j] = quantile(opt_boot[,j],0.025)
      length[j] = ub[j]-lb[j]
      sd_boot[j] = sd(opt_boot[,j])
      coverage[j] = c_opt[j]>=lb[j] && c_opt[j]<=ub[j]
      boot_results = c(length,sd_boot,coverage)
    }
    return(boot_results)
  }
  results=matrix(nrow=M,ncol = 6)
  bootstrap_results=matrix(nrow=M,ncol = 18)
  acc_results=matrix(nrow=M,ncol = 30)
  ##Relativebias and MSE
  rb=function(true,estimated){
    rb=(mean(estimated)-true)/true
    return(rb)
  }
  mse_f=function(true,estimated){
    mse_f=mean((estimated-true)^2)
    return(mse_f)
  }
  # Simulation
  set.seed(1)
  for (i in 1:M) {
    x_nondiseased = rgamma(n=n0,shape = alpha0, scale=beta0)
    x_diseased = rgamma(n=n1,shape=alpha1,scale=beta1)
    X=c(x_nondiseased,x_diseased)
    status = c(rep(0,n0),rep(1,n1))
    data = data.frame(cbind(X,status))
    closest_cz = cz(data)
    closest_er = er(data)
    closest_youden = youden(data)
    closest_ed = index_euclidian(data)
    closest_iu = index_union(data)
    closest_chi = chi_max(data)
    data$cz<-ifelse(data$X>=closest_cz,1,0)
    data$er<-ifelse(data$X>=closest_er,1,0)
    data$youden<-ifelse(data$X>=closest_youden,1,0)
    data$chi<-ifelse(data$X>=closest_chi,1,0)
    data$ed<-ifelse(data$X>=closest_ed,1,0)
    data$iu<-ifelse(data$X>=closest_iu,1,0)
    table_cz = table(data$cz,data$status)
    cz_acc = confusionMatrix(table_cz,positive = "1")
    cz_results = c(cz_acc$overall[1],cz_acc$byClass[1:4])
    table_er = table(data$er,data$status)
    er_acc = confusionMatrix(table_er,positive = "1")
    er_results = c(er_acc$overall[1],er_acc$byClass[1:4])
    table_youden = table(data$youden,data$status)
    youden_acc = confusionMatrix(table_youden,positive = "1")
    youden_results = c(youden_acc$overall[1],youden_acc$byClass[1:4])
    table_chi = table(data$chi,data$status)
    chi_acc = confusionMatrix(table_chi,positive = "1")
    chi_results = c(chi_acc$overall[1],chi_acc$byClass[1:4])
    table_ed = table(data$ed,data$status)
    ed_acc = confusionMatrix(table_ed,positive = "1")
    ed_results = c(ed_acc$overall[1],ed_acc$byClass[1:4])
    table_iu = table(data$iu,data$status)
    iu_acc = confusionMatrix(table_iu,positive = "1")
    iu_results = c(iu_acc$overall[1],iu_acc$byClass[1:4])
    bootstrap = boot(data=data,B=B,true_param = c_opt)
  # Bias & MSE
    results[i,1:6]=matrix(c(closest_cz,closest_er,closest_youden,closest_chi,
                            closest_iu,closest_ed),ncol=6,byrow=F)
    bootstrap_results[i,1:18]=matrix(bootstrap,ncol=18,byrow=F)
    acc_results[i,1:30]=matrix(c(cz_results,er_results,youden_results,chi_results,iu_results,ed_results),ncol=30,byrow=F)
  }  
colnames(results)=c("cz","er","youden","max_chi","iu","ed")
colnames(bootstrap_results) = c("cz_length","er_length","youden_length","maxp_length","iu_length","ed_lenght",
                                    "cz_sd","er_sd","youden_sd","maxp_sd","iu_sd","ed_sd","cz_cover","er_cover",
                                    "youden_cover","max_cover","iu_cover","ed_cover")
colnames(acc_results) = c("cz_acc","cz_se","cz_sp","cz_ppv","cz_npv","er_acc","er_se","er_sp",
                              "er_ppv","er_npv","youden_acc","youden_se","youden_sp","youden_ppv",
                              "youden_npv","chi_acc","chi_se","chi_sp","chi_ppv","chi_npv","iu_acc",
                              "iu_se","iu_sp","iu_ppv","iu_npv","ed_acc","ed_se","ed_sp","ed_ppv",
                           "ed_npv")
 bias=NULL
for(l in 1:6){
 bias[l] = rb(c_opt[l],results[,l])
 }
mse = NULL
for(k in 1:6){
mse[k]=mse_f(c_opt[k],results[,k])
}
    all_perf_results=t(c(bias,mse))
    all_perf_results=data.frame(all_perf_results)
    colnames(all_perf_results)=c("cz_bias","er_bias","youden_bias","chi_bias","iu_bias","ed_bias",
                                 "cz_mse","er_mse","youden_mse","chi_mse","iu_mse","ed_mse")
    
 
  
  return(list(
    cutoffs = results,
    bootstrap = bootstrap_results,
    accuracy = acc_results,
    performance = all_perf_results,
    params = data.frame(n1 = n1, n0 = n0, mu1 = mu1),
    true_cuts=c_opt
  ))
}
s25=run_simulation_gamma(n0=50,n1=50,beta1=0.79,M=1000,B=200)
s25_results=data.frame(cbind(s25$params,dist="gamma",s25$cutoffs))
s25_perf=data.frame(cbind(s25$params,dist="gamma",s25$performance))
s25_acc=data.frame(cbind(s25$params,dist="gamma",s25$accuracy))
s25_boot=data.frame(cbind(s25$params,dist="gamma",s25$bootstrap))
#########
s26=run_simulation_gamma(n0=100,n1=100,beta1=0.79,M=1000,B=200)
s26_results=data.frame(cbind(s26$params,dist="gamma",s26$cutoffs))
s26_perf=data.frame(cbind(s26$params,dist="gamma",s26$performance))
s26_acc=data.frame(cbind(s26$params,dist="gamma",s26$accuracy))
s26_boot=data.frame(cbind(s26$params,dist="gamma",s26$bootstrap))
#########
s27=run_simulation_gamma(n0=200,n1=200,beta1=0.79,M=1000,B=200)
s27_results=data.frame(cbind(s27$params,dist="gamma",s27$cutoffs))
s27_perf=data.frame(cbind(s27$params,dist="gamma",s27$performance))
s27_acc=data.frame(cbind(s27$params,dist="gamma",s27$accuracy))
s27_boot=data.frame(cbind(s27$params,dist="gamma",s27$bootstrap))
#########
s28=run_simulation_gamma(n0=50,n1=50,beta1=1.22,M=1000,B=200)
s28_results=data.frame(cbind(s28$params,dist="gamma",s28$cutoffs))
s28_perf=data.frame(cbind(s28$params,dist="gamma",s28$performance))
s28_acc=data.frame(cbind(s28$params,dist="gamma",s28$accuracy))
s28_boot=data.frame(cbind(s28$params,dist="gamma",s28$bootstrap))
#####
s29=run_simulation_gamma(n0=100,n1=100,beta1=1.22,M=1000,B=200)
s29_results=data.frame(cbind(s29$params,dist="gamma",s29$cutoffs))
s29_perf=data.frame(cbind(s29$params,dist="gamma",s29$performance))
s29_acc=data.frame(cbind(s29$params,dist="gamma",s29$accuracy))
s29_boot=data.frame(cbind(s29$params,dist="gamma",s29$bootstrap))
#######
s30=run_simulation_gamma(n0=200,n1=200,beta1=1.22,M=1000,B=200)
s30_results=data.frame(cbind(s30$params,dist="gamma",s30$cutoffs))
s30_perf=data.frame(cbind(s30$params,dist="gamma",s30$performance))
s30_acc=data.frame(cbind(s30$params,dist="gamma",s30$accuracy))
s30_boot=data.frame(cbind(s30$params,dist="gamma",s30$bootstrap))
###
s31=run_simulation_gamma(n0=50,n1=50,beta1=1.97,M=1000,B=200)
s31_results=data.frame(cbind(s31$params,dist="gamma",s31$cutoffs))
s31_perf=data.frame(cbind(s31$params,dist="gamma",s31$performance))
s31_acc=data.frame(cbind(s31$params,dist="gamma",s31$accuracy))
s31_boot=data.frame(cbind(s31$params,dist="gamma",s31$bootstrap))
#########
s32=run_simulation_gamma(n0=100,n1=100,beta1=1.97,M=1000,B=200)
s32_results=data.frame(cbind(s32$params,dist="gamma",s32$cutoffs))
s32_perf=data.frame(cbind(s32$params,dist="gamma",s32$performance))
s32_acc=data.frame(cbind(s32$params,dist="gamma",s32$accuracy))
s32_boot=data.frame(cbind(s32$params,dist="gamma",s32$bootstrap))
##########
#########
s33=run_simulation_gamma(n0=200,n1=200,beta1=1.97,M=1000,B=200)
s33_results=data.frame(cbind(s33$params,dist="gamma",s33$cutoffs))
s33_perf=data.frame(cbind(s33$params,dist="gamma",s33$performance))
s33_acc=data.frame(cbind(s33$params,dist="gamma",s33$accuracy))
s33_boot=data.frame(cbind(s33$params,dist="gamma",s33$bootstrap))
#########
s34=run_simulation_gamma(n0=50,n1=50,beta1=3.82,M=1000,B=200)
s34_results=data.frame(cbind(s34$params,dist="gamma",s34$cutoffs))
s34_perf=data.frame(cbind(s34$params,dist="gamma",s34$performance))
s34_acc=data.frame(cbind(s34$params,dist="gamma",s34$accuracy))
s34_boot=data.frame(cbind(s34$params,dist="gamma",s34$bootstrap))
#########
s35=run_simulation_gamma(n0=100,n1=100,beta1=3.82,M=1000,B=200)
s35_results=data.frame(cbind(s35$params,dist="gamma",s35$cutoffs))
s35_perf=data.frame(cbind(s35$params,dist="gamma",s35$performance))
s35_acc=data.frame(cbind(s35$params,dist="gamma",s35$accuracy))
s35_boot=data.frame(cbind(s35$params,dist="gamma",s35$bootstrap))
#######
s36=run_simulation_gamma(n0=200,n1=200,beta1=3.82,M=1000,B=200)
s36_results=data.frame(cbind(s36$params,dist="gamma",s36$cutoffs))
s36_perf=data.frame(cbind(s36$params,dist="gamma",s36$performance))
s36_acc=data.frame(cbind(s36$params,dist="gamma",s36$accuracy))
s36_boot=data.frame(cbind(s36$params,dist="gamma",s36$bootstrap))
